In [2]:
from transformers import AutoModel, AutoTokenizer
import torch
import numpy as np
import random
import pandas as pd
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
import copy
import math
import torch.nn.functional as F
import joblib

In [3]:
def set_seed(seed):
    """PyTorch随机数种子设置大全"""
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)  # CPU上设置随机种子
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)  # 当前GPU上设置随机种子
        # torch.cuda.manual_seed_all(seed) # 所有GPU上设置随机种子


seed = 42
set_seed(seed)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
df_test = pd.read_csv("../../datasets/test.csv")
df_test.head()

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...


In [6]:
class MyDataset(Data.Dataset):
    """定义数据集"""

    def __init__(self, df, have_target=True):
        self.dataset = df
        self.have_target = have_target

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        """定义索引方式"""
        text = self.dataset.iloc[i]['excerpt']
        if self.have_target:
            target = self.dataset.iloc[i]['target']
            return text, target
        else:
            return text,


data_test = MyDataset(df_test, have_target=False)

for text, in data_test:
    # 调用__getitem__方法
    print(text)
    break

My hope lay in Jack's promise that he would keep a bright light burning in the upper story to guide me on my course. On a clear night this light was visible from the village, but somehow or other I failed to take into account the state of the weather. The air was full of eddying flakes, which would render the headlight of a locomotive invisible a hundred yards distant. Strange that this important fact never occurred to me until I was fully a fourth of a mile from the village. Then, after looking in vain for the beacon light, the danger of my situation struck me, and I halted.
"I am certain to go wrong," I said to myself.
"It is out of my power to follow a direct course without something to serve as a compass. I will go back to the village and wait till morning."


In [7]:
model_name = 'roberta-large'

tokenizer = AutoTokenizer.from_pretrained(model_name)
print(tokenizer.model_input_names)

pretrained = AutoModel.from_pretrained('../继续训练预训练模型_roberta-large/save_model/')
print(pretrained.num_parameters())  # 参数量

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

['input_ids', 'attention_mask']


Some weights of the model checkpoint at ../继续训练预训练模型_roberta-large/save_model/ were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../继续训练预训练模型_roberta-large/save_model/ and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predict

355359744


In [9]:
def get_collate_fn(tokenizer, max_len=256):
    """返回collate_fun函数(通过闭包函数引入形参)"""

    def collate_fn(data):
        sents = [i[0] for i in data]

        # 批量编码句子
        text_t = tokenizer(text=sents,
                           truncation=True,
                           padding='max_length',
                           max_length=max_len,
                           return_token_type_ids=True,
                           return_attention_mask=True,
                           return_tensors='pt')

        input_ids = text_t['input_ids']
        attention_mask = text_t['attention_mask']
        if len(data[0]) == 1:
            return input_ids, attention_mask
        else:
            target = torch.tensor([i[1] for i in data], dtype=torch.float32)
            return input_ids, attention_mask, target

    return collate_fn


dataloader_test = torch.utils.data.DataLoader(dataset=data_test, batch_size=8, collate_fn=get_collate_fn(tokenizer),
                                              shuffle=False)

for input_ids, attention_mask, in dataloader_test:
    print(input_ids)
    print(attention_mask)
    break

tensor([[    0,  2387,  1034,  ...,     1,     1,     1],
        [    0,   495,  1242,  ...,     1,     1,     1],
        [    0,   243,    21,  ...,     1,     1,     1],
        ...,
        [    0, 49714,  3923,  ...,     1,     1,     1],
        [    0,  3972,  3922,  ...,     1,     1,     1],
        [    0, 28151,  2348,  ...,     1,     1,     1]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [10]:
class MyModel(nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.pretrained = pretrained_model
        self.norm = nn.LayerNorm(1024)  # roberta-large隐藏层大小为1024
        self.linear = nn.Linear(1024, 1)

    def forward(self, input_ids, attention_mask):
        emb = self.pretrained(input_ids, attention_mask=attention_mask)["last_hidden_state"]
        emb = torch.mean(emb, dim=1)
        output = self.norm(emb)
        output = self.linear(output)
        return output

In [11]:
def get_parameters(model,
                   encoder_layer_init_lr=2e-5,  # bert模型最后一个encoder结构的学习率
                   multiplier=0.95,  # 衰退因子
                   custom_lr=1e-4):  # 自定义的网络层学习率
    parameters = []
    lr = encoder_layer_init_lr

    # encoder层:
    # bert-larger共有24个encoder结构(分别为encoder.layer.0, encoder.layer.1, ......, encoder.layer.23)
    # bert-base共有12个encoder结构(分别为encoder.layer.0, encoder.layer.1, ......, encoder.layer.11)
    for layer in range(23, -1, -1):
        layer_params = {
            'params': [param for name, param in model.named_parameters() if f'encoder.layer.{layer}.' in name],
            'lr': lr
        }
        parameters.append(layer_params)
        lr *= multiplier  # 上个encoder结构的学习率 = 该encoder结构的学习率 * 衰退因子

    # embedding层:bert模型embedding层(最底层)
    embeddings_params = {
        'params': [param for name, param in model.named_parameters() if 'embeddings' in name],  # 关键字in表示是否包含
        'lr': 1e-7
    }
    parameters.append(embeddings_params)

    # 自定义网络层:下游任务自定义的网络层(具体任务对应修改)
    custom_params = {
        'params': [param for name, param in model.named_parameters() if 'linear' in name or 'norm' in name],
        'lr': custom_lr
    }
    parameters.append(custom_params)
    return parameters

In [12]:
def get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5):
    """
    Create a schedule with a learning rate that decreases following the values of the cosine function between the
    initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
    initial lr set in the optimizer.
    Args:
        optimizer ([`~torch.optim.Optimizer`]):
            The optimizer for which to schedule the learning rate.
        num_warmup_steps (`int`):
            The number of steps for the warmup phase.
        num_training_steps (`int`):
            The total number of training steps.
        num_cycles (`float`, *optional*, defaults to 0.5):
            The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
            following a half-cosine).
    Return:
        `torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
    """

    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            # 学习率预热(线性增加)
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))  # 根据cos函数变化

    return LambdaLR(optimizer, lr_lambda)

In [13]:
# 模型验证
def evaluate(model, dataloader, device):
    model.eval()

    loss_7 = []
    with torch.no_grad():
        for input_ids, attention_mask, target in dataloader:
            # 数据设备切换
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            out = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = torch.sqrt(F.mse_loss(out.cpu().reshape(-1), target, reduction='mean'))
            loss_7.append(loss.item())

    return torch.mean(torch.tensor(loss_7)).item()  # 平均rmse


# 每训练10个step进行一次模型验证
def train_and_evaluate(model, dataloader_train, dataloader_val, criterion, optimizer, scheduler_lr, epochs, device):
    model.train()

    best_valid_rmse = 1e7  # 最佳模型验证数据集的rmse
    best_model_state_dict = [None]  # 最佳模型的状态字典
    for epoch in range(1, epochs + 1):
        for idx, (input_ids, attention_mask, target) in enumerate(dataloader_train):
            # 数据设备切换
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            out = model(input_ids=input_ids, attention_mask=attention_mask)
            out = out.reshape(-1)

            loss = criterion(out, target)  # 每个step的损失值

            loss.backward()
            optimizer.step()
            scheduler_lr.step()

            if idx % 10 == 0 and idx > 0:  # 每10 step进行一次模型验证
                val_rmse = evaluate(model, dataloader_val, device)
                if val_rmse < best_valid_rmse:
                    best_valid_rmse = val_rmse
                    best_model_state_dict.pop()
                    best_model_state_dict.append(copy.deepcopy(model.state_dict()))  # 状态字典必须进行深拷贝
                    print('| end of epoch {:5d} | step: {:5d} | valild rmse {:8.5f} |'.format(epoch, idx, val_rmse))

    return best_valid_rmse, best_model_state_dict

In [17]:
data_oof = pd.read_csv("../../data/df_train_oof.csv", index_col=0)  # 加载分层5折数据集
data_oof.head()

,id,url_legal,license,excerpt,target,standard_error,fold
0,b51730f9c,NaN,NaN,"Alice looked at the jury-box, and saw that, in...",-0.432678,0.487498,0
1,4d403fd57,https://en.wikipedia.org/wiki/Artificial_intel...,CC BY-SA 3.0,Artificial intelligence (AI) is intelligence e...,-1.161746,0.458396,2
2,0f789ee41,NaN,NaN,A gruff squire on horseback with shiny top boo...,-2.367914,0.519369,4
3,87f96eb79,NaN,NaN,But that hadn't helped Washington.\nThe Americ...,-0.842596,0.466193,0
4,b9cca6661,NaN,NaN,The principal business of the people of this c...,-0.748452,0.433000,4


In [18]:
fold_num_list = range(5)

best_valid_rmse_lst = []  # 每折交叉验证最佳模型验证数据集的rmse
best_model_state_dict_lst = []  # 每折交叉验证最佳模型的状态字典

for fold in fold_num_list:
    print('*' * 40 + str(fold) + '*' * 40)

    train_data = data_oof[data_oof['fold'] != fold]  # 训练数据集
    val_data = data_oof[data_oof['fold'] == fold]  # 验证数据集
    dataloader_train = torch.utils.data.DataLoader(dataset=MyDataset(train_data), batch_size=8,
                                                   collate_fn=get_collate_fn(tokenizer), shuffle=True)
    dataloader_val = torch.utils.data.DataLoader(dataset=MyDataset(val_data),
                                                 # 测试数据只有7条((\sqrt{(n1 + n2)/2} + \sqrt{(n3 + n4)/2} + \sqrt{(n5 + n6)/2}) / 3 不等于\sqrt{(n1 + n2 + n3 + n4 + n5 + n6)/6})
                                                 batch_size=7,
                                                 collate_fn=get_collate_fn(tokenizer),
                                                 shuffle=False, drop_last=True)

    # **************************************************************************************
    roberta_large = MyModel(copy.deepcopy(pretrained))  # 必须进行深拷贝(pretrained会参与更新),否则会造成标签泄露
    roberta_large = roberta_large.to(device)

    loss_mse = nn.MSELoss()

    parameters = get_parameters(roberta_large, 2e-5, 0.95, 1e-4)
    # 优化器
    optimizer_adamw = optim.AdamW(parameters)
    scheduler_lr = get_cosine_schedule_with_warmup(optimizer_adamw, 0, len(dataloader_train) * 5)
    # **************************************************************************************

    bvr, bmsd = train_and_evaluate(roberta_large, dataloader_train, dataloader_val, loss_mse, optimizer_adamw,
                                   scheduler_lr, 5, device)

    best_valid_rmse_lst.append(bvr)
    best_model_state_dict_lst.extend(bmsd)

****************************************0****************************************
| end of epoch     1 | step:    10 | valild rmse  0.99068 |
| end of epoch     1 | step:    20 | valild rmse  0.87099 |
| end of epoch     1 | step:    30 | valild rmse  0.78200 |
| end of epoch     1 | step:    40 | valild rmse  0.72259 |
| end of epoch     1 | step:    50 | valild rmse  0.61960 |
| end of epoch     1 | step:    70 | valild rmse  0.58218 |
| end of epoch     1 | step:    90 | valild rmse  0.56217 |
| end of epoch     1 | step:   100 | valild rmse  0.55258 |
| end of epoch     1 | step:   130 | valild rmse  0.54190 |
| end of epoch     1 | step:   200 | valild rmse  0.53571 |
| end of epoch     1 | step:   220 | valild rmse  0.53357 |
| end of epoch     1 | step:   230 | valild rmse  0.49138 |
| end of epoch     2 | step:    20 | valild rmse  0.47190 |
| end of epoch     2 | step:    30 | valild rmse  0.46850 |
| end of epoch     3 | step:    10 | valild rmse  0.46512 |
| end of epoch    

In [24]:
# 模型预测
def predict(model, dataloader, device):
    model.eval()

    predict_list = []
    with torch.no_grad():
        for i in dataloader:
            # 数据设备
            input_ids = i[0].to(device)
            attention_mask = i[1].to(device)
            out = model(input_ids=input_ids, attention_mask=attention_mask)
            predict_list.append(out.cpu())
    predict_all = torch.cat(predict_list, dim=0)
    return predict_all

In [25]:
all_tr_predict = torch.tensor([]).reshape((0, 1))
all_te_predict = torch.tensor([]).reshape((0, 1))

for fold in fold_num_list:
    val_d = data_oof[data_oof['fold'] == fold]
    dl_val = torch.utils.data.DataLoader(dataset=MyDataset(val_d), batch_size=8, collate_fn=get_collate_fn(tokenizer))
    # **************************************************************************************
    roberta_large = MyModel(copy.deepcopy(pretrained))  # 必须进行深拷贝(pretrained会参与更新),否则会造成标签泄露
    roberta_large.load_state_dict(best_model_state_dict_lst[fold])  # 记载状态字典
    roberta_large = roberta_large.to(device)
    # **************************************************************************************
    predict_result_tr = predict(roberta_large, dl_val, device)
    all_tr_predict = torch.cat([all_tr_predict, predict_result_tr])

    predict_result_te = predict(roberta_large, dataloader_test, device)
    all_te_predict = torch.cat([all_te_predict, predict_result_te])

In [26]:
all_tr_predict.shape, all_te_predict.shape

(torch.Size([2834, 1]), torch.Size([35, 1]))

In [27]:
joblib.dump(all_tr_predict.numpy(), 'roberta_large_tr.pkl')

['roberta_large_tr.pkl']

In [28]:
joblib.dump(all_te_predict.numpy(), 'roberta_large_te.pkl')

['roberta_large_te.pkl']